In [23]:
import numpy as np
np.random.seed(0)

# Log plot scale
def xlog_scale(log_x_max, scale, log_base= 10): 
    '''Logaritmic scale up to log_alpha_max'''
    bd_block = np.arange(0, log_base**2, log_base) + log_base
    bd_block = bd_block[0:-1]
    xlog = np.tile(bd_block, log_x_max)
    xlog[(log_base-1) : 2*(log_base-1)] = log_base*xlog[(log_base-1) : 2*(log_base-1)]
    for j in range(1, log_x_max - 1):
        xlog[(j+1)*(log_base-1) : (j+2)*(log_base-1)] = log_base*xlog[  j*(log_base-1) :  (j+1)*(log_base-1)  ]
    xlog = np.insert(xlog, 0,  np.arange(1,log_base), axis=0)
    xlog = np.insert(xlog, len(xlog),log_base**(log_x_max+1), axis=0)
    jlog = (xlog*scale).astype(int)
    return jlog

### Build Cython code

If not done before, build the cython code on the command line.

In [11]:
! python setup.py build_ext --inplace

running build_ext


### Import simulation class

In [12]:
from trainSGD import trainSGD

### Parameters

In [20]:
# Data dimension
d = 100
# Number of data points
n = 40
# Choose p 
p = 10
# Number of S random instances
nS = 10
# Signal-to-noise ratio
snr = 1./5.
# Max number of (S)GD iterations
n_it_max= int(1e6)  
##### Number of SGD runs 
n_SGD = 1
##### Store only the terminal point in the dynamics
only_end = False
##### Learning rate
lr = 1./d
##### Data matrix
X = np.random.randn(n,d)
##### Beta star d-dimension (ground true)
beta_star_ = np.random.randn(d)
beta_star = np.copy(beta_star_) / np.linalg.norm(beta_star_)
###### Initializing the estimator
beta0_ = np.random.randn(d)
beta0  = np.copy(beta0_)/np.linalg.norm(beta0_)
##### Targets
y = X @ beta_star + snr*np.random.randn(n)
#### Plot
log_x_max = (np.log10(n_it_max)-1).astype(int)  
plot_list = xlog_scale(log_x_max, scale=1., log_base= 10)
if only_end:
    plot_list = plot_list[-1:]
plot_list = np.insert(plot_list, 0, 0)

In [21]:
print(plot_list)
print(plot_list.shape)

[      0       1       2       3       4       5       6       7       8
       9      10      20      30      40      50      60      70      80
      90     100     200     300     400     500     600     700     800
     900    1000    2000    3000    4000    5000    6000    7000    8000
    9000   10000   20000   30000   40000   50000   60000   70000   80000
   90000  100000  200000  300000  400000  500000  600000  700000  800000
  900000 1000000]
(56,)


### Training

In [15]:
betaSGD = trainSGD(p=p, nS=nS, lr=lr, X=X, y=y, beta0= beta0, plot_list=plot_list, n=n, d=d, n_it_max=n_it_max, n_SGD= n_SGD)

Beginning SGD training ----- p= 10 --- nS= 10
Time elapsed SGD: 26.183378
End ----- p= 10 --- nS= 10
 


In [16]:
betaSGD

array([[ 0.0888479 , -0.00881707, -0.02864405,  0.17181028,  0.01885469,
         0.02708222, -0.02105735,  0.11259052,  0.02445633, -0.10588934],
       [ 0.09090779, -0.00806358, -0.03152908,  0.17882402,  0.02109943,
         0.03481068, -0.01440968,  0.12016649,  0.01044891, -0.10535464],
       [ 0.10926683,  0.01862255,  0.00258448,  0.17638279,  0.03854931,
         0.04953575, -0.04575601,  0.13676799,  0.0071859 , -0.10566546],
       [ 0.10543426,  0.02061127,  0.00560334,  0.17280863,  0.03169273,
         0.04996029, -0.05796531,  0.13514624, -0.00244581, -0.10470284],
       [ 0.106244  ,  0.01982098,  0.01435461,  0.18178964,  0.00436922,
         0.07844776, -0.07964515,  0.13949753, -0.00640101, -0.08300208],
       [ 0.09424177,  0.0179098 ,  0.01842961,  0.18406404,  0.00163376,
         0.07032126, -0.07410687,  0.13299849, -0.00558801, -0.08726391],
       [ 0.08912019,  0.01485751,  0.01698422,  0.15649277, -0.00448352,
         0.06524365, -0.06618467,  0.13132842

In [22]:
print(betaSGD.shape)

(56, 10)


### In the paper

To get the training weights used in the paper, use the script `scrSGD.py` after building the Cython code. One might want to appropriately set `intI` and `intF` to get a particular interval in `alpha`. 

In [ ]:
! python scrSGD.py